In [1]:
# загрузите необходимые библиотеки
import pandas as pd
import numpy as np

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(legacy="1.25")

In [4]:
titanic = pd.read_csv('data/titanic.csv', index_col='PassengerId')

In [5]:
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Задание 1

В датасете Titanic проверьте признак "Возраст"("Age") на выбросы (отрицательный возраст, посмотрите максимальный возраст - он правдоподобен?).  
Если там есть отсутствующие значения - на их место поставьте медианный возраст пассажиров.

In [7]:
titanic.Age.min()
titanic.Age.max()
titanic.Age.describe()
# negative age check
negative_age = titanic[titanic['Age'] < 0]
over_age = titanic[titanic.Age > 100]
print(f"negative age exists: {not negative_age.empty}")
print(f"over age exists: {not over_age.empty}")

0.42

80.0

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

negative age exists: False
over age exists: False


In [8]:
print("Before:")
missing_age = titanic['Age'].isna().sum()
print(f"missed_age: {missing_age}")
age_median = titanic.Age.median
print(f'Median: {age_median()}')

Before:
missed_age: 177
Median: 28.0


In [9]:
titanic['Age'] = titanic.Age.fillna(age_median())

In [10]:
print("After filling:")
missing_age = titanic['Age'].isna().sum()
print(f"missed_age: {missing_age}")
print(f'Median: {age_median()}')

After filling:
missed_age: 0
Median: 28.0


---

### Задание 2
Проанализируйте загруженные данные:

1. Сколько было на Титанике мужчин, женщин и несовершеннолетних детей(<18 лет)?
2. Действительно ли у пассажиров, путешествующих 3-м классом нет номеров кают?
3. Есть предположение, что женщин и детей не было среди путешествующих 3-м классом. Проверьте.

### Решения

1. Сколько было на Титанике мужчин, женщин и несовершеннолетних детей(<18 лет)?

In [12]:
children = titanic.query("Age < 18").index.size 
men = titanic.query("Age >= 18 and Sex == 'male'").shape[0]
women = titanic.query("Age >= 18 and Sex == 'female'").Name.count()
print(f'{children = }, {men = }, {women = }')

adults = titanic.query("Age >= 18")
adults.Sex.value_counts()

children = 113, men = 519, women = 259


Sex
male      519
female    259
Name: count, dtype: int64

2. Действительно ли у пассажиров, путешествующих 3-м классом нет номеров кают?

In [16]:
# Пример работы с np.nan в pandas.query()
titanic.query('Pclass == 3 and Cabin == Cabin').Cabin.to_frame()
# np.nan == np.nan # False

,Cabin
PassengerId,
11,G6
76,F G73
129,F E69
206,G6
252,G6
395,G6
430,E10
700,F G63
716,F G73


In [18]:
titanic.query('Pclass == 3').Cabin.isna().all()
cabin_stats = titanic.query('Pclass == 3 & Cabin.notna()').Cabin.count()
cabin_stats

False

12

In [19]:
titanic.query('Pclass == 3').Cabin.unique()

array([nan, 'G6', 'F G73', 'F E69', 'E10', 'F G63', 'E121', 'F38'],
      dtype=object)

In [20]:
titanic.query('Pclass == 3').dropna().Cabin.to_frame().empty

False

3. Есть предположение, что женщин и детей не было среди путешествующих 3-м классом. Проверьте.

In [21]:
print("woman in 3d class", titanic.query('Pclass == 3 and Sex == "female" and Age >= 18').shape[0])
print("kids <18 in 3d class", titanic.query('Pclass == 3 and Age < 18').index.size)
print("woman or kids in 3d class", titanic.query('Pclass == 3 and (Sex == "female" | Age < 18)').shape[0])

woman in 3d class 109
kids <18 in 3d class 78
woman or kids in 3d class 187


In [ ]:
# Ваш код